In [16]:
!pip install python-chess
!pip install psutil

import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import glob
import os
import chess


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

!pip install kaggle
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
!kaggle datasets download -d mallikarjunreddy3015/chessfendataset

 94% 136M/144M [00:02<00:00, 115MB/s] 
100% 144M/144M [00:02<00:00, 70.9MB/s]


In [ ]:
import psutil

# Get system memory information
memory = psutil.virtual_memory()
print(f"Total RAM: {memory.total / (1024 ** 3):.2f} GB")
print(f"Used RAM: {memory.used / (1024 ** 3):.2f} GB")
print(f"Free RAM: {memory.free / (1024 ** 3):.2f} GB")


Total RAM: 56.49 GB
Used RAM: 1.73 GB
Free RAM: 52.92 GB


In [ ]:
import zipfile

# Replace 'your-dataset.zip' with the actual zip file name
with zipfile.ZipFile('/content/chessfendataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/chessfendataset')


In [ ]:
dataframe = pd.read_csv('/content/chessfendataset/chessData.csv')

In [ ]:
dataframe.head()
d=dataframe

In [ ]:
d.shape
d1=d.iloc[2000000:4000000  ]
print(d1.shape)
d1 = d1[~d1['Evaluation'].str.contains('\ufeff|#', na=False)]
d1['Evaluation'] = pd.to_numeric(d1['Evaluation'].str.replace('[^\d.-]', '', regex=True), errors='coerce')

(2000000, 2)


In [ ]:
d1['Scaled_Evaluation'] = 1000 * (2 * (d1['Evaluation'] +7000) / (14000) - 1)

In [ ]:
import chess
import numpy as np
i=0
def create_bitboard(fen):
    global i
    i+=1
    # Setting up the piece values
    piece_values = {
        'P': 11, 'p': -11, 'N': 44, 'n': -44, 'B': 33, 'b': -33, 'R': 55,
        'r': -55, 'Q': 99, 'q': -99, 'K': 110, 'k': -110
    }

    # Mapping piece symbols to bitboard layers
    piece_to_layer = {
        'P': 0, 'p': 1, 'N': 2, 'n': 3, 'B': 4, 'b': 5, 'R': 6, 'r': 7, 'Q': 8,
        'q': 9, 'K': 10, 'k': 11
    }

    # Create an empty bitboard of shape (12, 8, 8) using numpy
    en_passant_layer = 12
    white_values_layer = 13
    black_values_layer = 14
    valid_move_positions_layer = 15

    bitboard = np.zeros((16, 8, 8))

    board = chess.Board(fen)
    if board.ep_square is not None:
        rank, file = divmod(board.ep_square, 8)
        bitboard[en_passant_layer][rank][file] = 1 if board.turn == chess.WHITE else -1
    computed_moves = {}

    for piece_type, piece_value in piece_values.items():
        layer = piece_to_layer[piece_type]

        if piece_type not in computed_moves:
            valid_moves = set()
            empty_board_moves = set()
            for square in chess.SQUARES:
                piece = board.piece_at(square)
                if piece and piece.symbol() == piece_type:
                    # Generate moves for both colors irrespective of turn
                    board.turn = chess.WHITE
                    moves_white = [move for move in board.legal_moves if move.from_square == square]
                    board.turn = chess.BLACK
                    moves_black = [move for move in board.legal_moves if move.from_square == square]
                    valid_moves.update([move.to_square for move in moves_white])
                    valid_moves.update([move.to_square for move in moves_black])

                    empty_board = chess.Board()
                    empty_board.clear()
                    empty_board.set_piece_at(square, piece)
                    # Generate moves for both colors irrespective of turn on the empty board
                    empty_board.turn = chess.WHITE
                    empty_moves_white = [move for move in empty_board.generate_legal_moves(from_mask=chess.BB_SQUARES[square])]
                    empty_board.turn = chess.BLACK
                    empty_moves_black = [move for move in empty_board.generate_legal_moves(from_mask=chess.BB_SQUARES[square])]
                    empty_board_moves.update([move.to_square for move in empty_moves_white])
                    empty_board_moves.update([move.to_square for move in empty_moves_black])

            computed_moves[piece_type] = (valid_moves, empty_board_moves)

        valid_moves, empty_board_moves = computed_moves[piece_type]

        for square in chess.SQUARES:
            rank, file = divmod(square, 8)
            piece = board.piece_at(square)
            if piece:
                if piece.color == chess.WHITE:
                    bitboard[white_values_layer][rank][file] = piece_values[piece.symbol()]
                else:
                    bitboard[black_values_layer][rank][file] = piece_values[piece.symbol()]
            if piece and piece.symbol() == piece_type:
                bitboard[layer][rank][file] = piece_value
            elif square in valid_moves:
                bitboard[layer][rank][file] = piece_value / 11
            elif square in empty_board_moves:
                bitboard[layer][rank][file] = piece_value / 22
        board = chess.Board(fen)
        for move in board.legal_moves:
            rank, file = divmod(move.from_square, 8)
            if board.turn == chess.WHITE:
              bitboard[valid_move_positions_layer][rank][file] = 1
            else:
              bitboard[valid_move_positions_layer][rank][file] = -1
    if(i%100000==0):
      print(i)
    return bitboard

bitboard_dataset = [create_bitboard(fen) for fen in np.array(d1['FEN'])]
# print(create_bitboard(np.array(d1["FEN"])[1])[15])


100000


In [ ]:
print(bitboard_dataset[20])

In [ ]:
# labels=d1['Evaluation']
labels=d1['Scaled_Evaluation']

In [ ]:
np.save('/content/bitboard_dataset_2-4M.npy', bitboard_dataset)
np.save('/content/labels_dataset_2-4M.npy', labels)

# # bitboard_dataset=np.load("/content/chess-numpy-dataset/bitboard_dataset_1M.npy")
# # labels=np.load("/content/chess-numpy-dataset/labels_dataset.npy")

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/gleaming-modem-400906-3780658b0f47.json"
!pip install google-cloud-storage


from google.cloud import storage

def upload_to_bucket(blob_name, path_to_file, bucket_name):
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(path_to_file)
    print(f"File {path_to_file} uploaded to {blob_name}.")

# Use the function to upload your file:
upload_to_bucket("bitboard_dataset_2-4M.npy", "/content/bitboard_dataset_2-4M.npy", "chess_data_all")
upload_to_bucket("labels_dataset_2-4M.npy", "/content/labels_dataset_2-4M.npy", "chess_data_all")
